In [1]:
import sqlite3
from fastapi import FastAPI, Request, HTTPException
from fastapi.responses import JSONResponse
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel

In [2]:
from threading import Thread


In [3]:
app = FastAPI()

In [4]:
class Student(BaseModel):
    name: str
    course: str  
    code: str
    instructor: str

In [5]:

# Step 1: Connect to the SQLite database (creates the file if it doesn't exist)
conn = sqlite3.connect('database1.db')

# Step 2: Create a cursor object
cursor = conn.cursor()

# Step 3: Write SQL command to create a table
create_table_query = '''
CREATE TABLE IF NOT EXISTS students (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    course TEXT NOT NULL,
    code TEXT NOT NULL,
    instructor TEXT NOT NULL
);
'''

# Step 4: Execute the command
cursor.execute(create_table_query)

# Step 5: Commit the transaction
conn.commit()

# Step 6: Close the connection
conn.close()

print("Table created successfully!")

Table created successfully!


In [6]:
@app.post("/students/")
def InsertIntoTable(obj:Student) -> None:
    """
    this .... 
    """
    with sqlite3.connect('database1.db') as conn2: 
        curr = conn2.cursor()
        try:
            result = curr.execute('''
                INSERT INTO students (name, course, code, instructor) VALUES (?, ?, ?, ?)
            ''', (obj.name, obj.course, obj.code, obj.instructor))
        except NameError as e :
            print(f"Name Error Exception {e}")
        finally:
            print("finally executed")  
    # return GetTableInfo()
    
    

In [7]:
@app.get("/students/")
def GetTableInfo() -> list[tuple]:
    """
    return table results 
    """ 
    with sqlite3.connect("database1.db") as conn:
        cur = conn.cursor()
        try:
            results = cur.execute("""select * from students""")
            return results.fetchall()
        except Exception as e:
            print(f"Exception error {e}")

In [8]:
@app.put("/students/{student_id}", response_model=dict)
def PutUpdateStudent(student_id: int, obj: Student) -> None:
    """
    Update all fields of a specific student (PUT).
    """
    with sqlite3.connect('database1.db') as conn:
        curr = conn.cursor()
        try:
            curr.execute('''
                UPDATE students
                SET name = ?, course = ?, code = ?, instructor = ?
                WHERE id = ?
            ''', (obj.name, obj.course, obj.code, obj.instructor, student_id))
            conn.commit()
            print(f"Student with ID {student_id} updated successfully.")
        except Exception as e:
            print(f"Exception error: {e}")

In [9]:
# @app.patch("/students/{student_id}", response_model=dict)
def PatchUpdateStudent(student_id: int, **kwargs) -> None:
    """
    Update specific fields of a student (PATCH).
    Only updates fields provided in kwargs.
    """
    with sqlite3.connect('database1.db') as conn:
        curr = conn.cursor()
        # Generate dynamic SQL for the columns provided in kwargs
        fields = ", ".join(f"{key} = ?" for key in kwargs.keys())
        values = list(kwargs.values())
        values.append(student_id)
        try:
            query = f"UPDATE students SET {fields} WHERE id = ?"
            curr.execute(query, values)
            conn.commit()
            print(f"Student with ID {student_id} updated successfully.")
        except Exception as e:
            print(f"Exception error: {e}")

In [10]:
@app.delete("/students/{student_id}")
def DeleteStudent(student_id: int) -> None:
    """
    Delete a student record from the database.
    """
    with sqlite3.connect('database1.db') as conn:
        curr = conn.cursor()
        try:
            curr.execute('''
                DELETE FROM students
                WHERE id = ?
            ''', (student_id,))
            conn.commit()
            print(f"Student with ID {student_id} deleted successfully.")
        except Exception as e:
            print(f"Exception error: {e}")


In [11]:
def run_uvicorn():
    uvicorn.run(app, host="127.0.0.1", port=8000)

In [42]:
# std1 = Student(name="ahmed", course="bio", code="bio101", instructor="ali")

In [48]:
# std2 = Student(name="nora", course="math" , code="102" , instructor="omer")

In [59]:
# GetTableInfo()

[(2, 'ahmed', 'bio', 'bio101', 'ali'),
 (3, 'ahmed', 'bio', 'bio101', 'ali'),
 (4, 'smith', 'bio', '303', 'joen'),
 (5, 'smith', 'bio', '303', 'joen')]

In [46]:
# InsertIntoTable(std1)

finally executed


[(2, 'ahmed', 'bio', 'bio101', 'ali'), (3, 'ahmed', 'bio', 'bio101', 'ali')]

In [49]:
# PutUpdateStudent(1,std2)

Student with ID 1 updated successfully.


In [51]:
# PatchUpdateStudent(student_id=1, course="Chemistry")


Student with ID 1 updated successfully.


In [54]:
# DeleteStudent(student_id=1)


Student with ID 1 deleted successfully.


In [12]:
server_thread = Thread(target=run_uvicorn)
server_thread.start()

INFO:     Started server process [19088]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
